# Import Libraries

In [ ]:
!pip install category_encoders 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 2.6 MB/s eta 0:00:00


In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.9/212.9 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 10.3 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import math
import plotly.express as px

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import cut_tree
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
from mlxtend.preprocessing import minmax_scaling
from sklearn.preprocessing import minmax_scale
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer
import joblib

import category_encoders as ce
import optuna


In [ ]:
pandas.version

# Import Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/WorkFlow/OSX_DS_assignment.csv')
df = dataset.copy()
df

,user_name,country,review_title,review_description,designation,points,price,province,region_1,region_2,winery,variety
0,@kerinokeefe,Italy,Nicosia 2013 Vulkà Bianco (Etna),"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Nicosia,White Blend
1,@vossroger,Portugal,Quinta dos Avidagos 2011 Avidagos Red (Douro),"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Quinta dos Avidagos,Portuguese Red
2,@paulgwine,US,Rainstorm 2013 Pinot Gris (Willamette Valley),"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Rainstorm,Pinot Gris
3,NaN,US,St. Julian 2013 Reserve Late Harvest Riesling ...,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,St. Julian,Riesling
4,@paulgwine,US,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Sweet Cheeks,Pinot Noir
...,...,...,...,...,...,...,...,...,...,...,...,...
103317,NaN,Germany,Dr. H. Thanisch (Erben Müller-Burggraef) 2013 ...,Notes of honeysuckle and cantaloupe sweeten th...,Brauneberger Juffer-Sonnenuhr Spätlese,90,28.0,Mosel,NaN,NaN,Dr. H. Thanisch (Erben Müller-Burggraef),Riesling
103318,@paulgwine,US,Citation 2004 Pinot Noir (Oregon),Citation is given as much as a decade of bottl...,NaN,90,75.0,Oregon,Oregon,Oregon Other,Citation,Pinot Noir
103319,@vossroger,France,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,NaN,Domaine Gresser,Gewürztraminer
103320,@vossroger,France,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),"A dry style of Pinot Gris, this is crisp with ...",NaN,90,32.0,Alsace,Alsace,NaN,Domaine Marcel Deiss,Pinot Gris


In [ ]:
df = df.sample(frac=1).reset_index(drop=True)

Drop user_name & review_title as data is irrelevant


In [ ]:
df.drop('user_name', axis=1, inplace=True)

In [ ]:
df.drop('review_title', axis=1, inplace=True)

Drop region_2 as 57% is missing

In [ ]:
df.drop('region_2', axis=1, inplace=True)

In [ ]:
# Remove duplicate rows
df = df.drop_duplicates()

In [ ]:
df = df.dropna(subset=['country'])

In [ ]:
varieties = df['variety'].unique()
y = {variety:i for i, variety in enumerate(varieties)}

df['variety'] = df['variety'].replace(y)

In [ ]:
print(y)

{'Grüner Veltliner': 0, 'Zinfandel': 1, 'Tempranillo': 2, 'Nebbiolo': 3, 'Merlot': 4, 'Sauvignon Blanc': 5, 'Chardonnay': 6, 'Sparkling Blend': 7, 'Sangiovese': 8, 'Pinot Noir': 9, 'Rosé': 10, 'Riesling': 11, 'Malbec': 12, 'Portuguese Red': 13, 'Red Blend': 14, 'Syrah': 15, 'Cabernet Sauvignon': 16, 'Gewürztraminer': 17, 'Bordeaux-style White Blend': 18, 'Champagne Blend': 19, 'Cabernet Franc': 20, 'Rhône-style Red Blend': 21, 'White Blend': 22, 'Bordeaux-style Red Blend': 23, 'Pinot Gris': 24, 'Portuguese White': 25, 'Gamay': 26, 'Pinot Grigio': 27}


### **Country**

In [ ]:
for c in df['country'].unique():
  if (df['country'] == c).sum()<20:
    df = df[df['country'] != c]

print(df.shape)    


(95331, 9)


### **Province**

In [ ]:
df['province']=df['country']
# df['province'] = df['province'].fillna(df['country'])

<ipython-input-17-8abd206af406>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['province']=df['country']


In [ ]:
df

,country,review_description,designation,points,price,province,region_1,winery,variety
0,US,"This is a delicious, tongue-tickling wine, cri...",Ridgecrest Vineyards,91,24.0,US,Ribbon Ridge,Chehalem,0
1,US,An interesting Zin that attracts for its range...,Mustang Springs,90,36.0,US,Paso Robles,Peachy Canyon,1
2,Spain,Deep aromas of baked black fruits are bolstere...,Colección Privada,92,49.0,Spain,Rioja,Sierra Cantabria,2
3,Italy,Oddero's Barolo Bussia Soprana is laced with l...,Bussia Soprana,91,86.0,Italy,Barolo,Oddero,3
4,Chile,"Bright violet in color and well extracted, but...",Reserve,85,12.0,Chile,NaN,Cono Sur,4
...,...,...,...,...,...,...,...,...,...
103317,Germany,The mineralschiefer designation attests to the...,Mineralschiefer Trocken,90,29.0,Germany,NaN,Grans-Fassian,11
103318,Argentina,"Basic, fresh, honest aromas of red-berry fruit...",1300,87,NaN,Argentina,Mendoza,Andeluna,16
103319,Italy,"Made from Sangiovese, with 5% Merlot and 5% Ca...",NaN,86,25.0,Italy,Rosso di Montepulciano,Lunadoro,14
103320,Italy,"Aromas recall blue flower, tilled earth and a ...",NaN,88,29.0,Italy,Nebbiolo d'Alba,Albino Rocca,3


## **Region_1**

In [ ]:
df['region_1'] = df['region_1'].fillna(df['province'])

<ipython-input-19-2d56dc2733f3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['region_1'] = df['region_1'].fillna(df['province'])


## **Winery**

## **Designation**

In [ ]:
df['designation'] = df['designation'].fillna(df['winery'])

<ipython-input-20-6695f6e455b0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['designation'] = df['designation'].fillna(df['winery'])


## **Points**

In [ ]:
def assign_rating(points):
    if points >= 95:
        return 5
    elif points >= 80:
        return 4
    elif points >= 60:
        return 3
    elif points >= 40:
        return 2
    else:
        return 1

df['rating'] = df['points'].apply(assign_rating)

<ipython-input-21-3e3108970028>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rating'] = df['points'].apply(assign_rating)


## **Price**

In [ ]:
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1

# Define thresholds for outlier detection
lower_threshold = Q1 - 25.0* IQR
upper_threshold = Q3 + 25.0 * IQR

# Remove rows with outliers in the 'price' column
df = df[(df['price'] > lower_threshold) & (df['price'] < upper_threshold)]

In [ ]:
df['price'] = df['price'].fillna(df['price'].mean())


<ipython-input-23-6a1c23d09cd4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price'] = df['price'].fillna(df['price'].mean())


## **Review_description**

In [ ]:
keywords = ['tropical', 'fruity', 'dry', 'acid', 'ripe', 'tanni', 'berry', 'citrus', 'chocolate', 'cherry', 'lemon', 'oak', 'white', 'red', 'vanilla', 'herb', 'sweet', 'apple']
for keyword in keywords:
    df[keyword] = df['review_description'].apply(lambda x: 1 if keyword in x else 0)

<ipython-input-24-f13cd1aacc2c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[keyword] = df['review_description'].apply(lambda x: 1 if keyword in x else 0)
<ipython-input-24-f13cd1aacc2c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[keyword] = df['review_description'].apply(lambda x: 1 if keyword in x else 0)
<ipython-input-24-f13cd1aacc2c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

In [ ]:
df['sweet'] = df['review_description'].apply(lambda x: 0 if "not sweet" in x else 1 if "sweet" in x else 0)
df['apple'] = df['review_description'].apply(lambda x: 0 if ("pineapple" in x or "custard apple" in x) else 1 if "apple" in x else 0)
df['acid'] = df['review_description'].apply(lambda x: 0 if "not acid" in x else 1 if "acid" in x else 0)
df['dry'] = df['review_description'].apply(lambda x: 0 if "not dry" in x else 1 if "dry" in x else 0)
df['ripe'] = df['review_description'].apply(lambda x: 0 if "not ripe" in x else 1 if "ripe" in x else 0)

<ipython-input-25-c3895ef82587>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sweet'] = df['review_description'].apply(lambda x: 0 if "not sweet" in x else 1 if "sweet" in x else 0)
<ipython-input-25-c3895ef82587>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['apple'] = df['review_description'].apply(lambda x: 0 if ("pineapple" in x or "custard apple" in x) else 1 if "apple" in x else 0)
<ipython-input-25-c3895ef82587>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

In [ ]:
df.drop('review_description', axis=1, inplace=True)

<ipython-input-26-9fee6460e26f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop('review_description', axis=1, inplace=True)


In [ ]:
df

,country,designation,points,price,province,region_1,winery,variety,rating,tropical,...,chocolate,cherry,lemon,oak,white,red,vanilla,herb,sweet,apple
0,US,Ridgecrest Vineyards,91,24.0,US,Ribbon Ridge,Chehalem,0,4,0,...,0,0,0,0,0,0,0,0,0,1
1,US,Mustang Springs,90,36.0,US,Paso Robles,Peachy Canyon,1,4,0,...,0,0,0,0,0,0,0,1,1,0
2,Spain,Colección Privada,92,49.0,Spain,Rioja,Sierra Cantabria,2,4,0,...,1,0,1,1,0,1,0,0,0,0
3,Italy,Bussia Soprana,91,86.0,Italy,Barolo,Oddero,3,4,0,...,0,0,0,0,0,0,0,0,0,0
4,Chile,Reserve,85,12.0,Chile,Chile,Cono Sur,4,4,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103316,US,Conner Lee Vineyard,90,28.0,US,Columbia Valley (WA),àMaurice,6,4,0,...,0,0,0,0,0,1,0,0,0,1
103317,Germany,Mineralschiefer Trocken,90,29.0,Germany,Germany,Grans-Fassian,11,4,0,...,0,0,0,0,0,0,0,0,0,0
103319,Italy,Lunadoro,86,25.0,Italy,Rosso di Montepulciano,Lunadoro,14,4,0,...,0,0,0,0,0,0,1,0,0,0
103320,Italy,Albino Rocca,88,29.0,Italy,Nebbiolo d'Alba,Albino Rocca,3,4,0,...,0,1,0,0,0,0,0,0,1,0


## **Train & Test split**

In [ ]:
from sklearn.model_selection import train_test_split

# Assume that you have a dataset represented as a feature matrix X and a target vector y
y = df['variety']
X = df.drop('variety', axis=1)

# Split the data into a training set and a temporary test set using a 70/30 split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=0)

# Split the temporary test set into a validation set and a test set using a 50/50 split
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=0)

## **Preprocessing**

In [ ]:
def preprocess(df,pipe):
  df.drop('user_name', axis=1, inplace=True)
  df.drop('review_title', axis=1, inplace=True)
  df.drop('region_2', axis=1, inplace=True)

  df['province']=df['country']

  keywords = ['tropical', 'fruity', 'dry', 'acid', 'ripe', 'tanni', 'berry', 'citrus', 'chocolate', 'cherry', 'lemon', 'oak', 'white', 'red', 'vanilla', 'herb', 'sweet', 'apple']
  for keyword in keywords:
      df[keyword] = df['review_description'].apply(lambda x: 1 if keyword in x else 0)
  df['sweet'] = df['review_description'].apply(lambda x: 0 if "not sweet" in x else 1 if "sweet" in x else 0)
  df['apple'] = df['review_description'].apply(lambda x: 0 if ("pineapple" in x or "custard apple" in x) else 1 if "apple" in x else 0)
  df['acid'] = df['review_description'].apply(lambda x: 0 if "not acid" in x else 1 if "acid" in x else 0)
  df['dry'] = df['review_description'].apply(lambda x: 0 if "not dry" in x else 1 if "dry" in x else 0)
  df['ripe'] = df['review_description'].apply(lambda x: 0 if "not ripe" in x else 1 if "ripe" in x else 0)    

  df.drop('review_description', axis=1, inplace=True)


  df=pipe.transform(df)






In [ ]:
X_train.isnull().sum()

country        0
designation    0
points         0
price          0
province       0
region_1       0
winery         0
rating         0
tropical       0
fruity         0
dry            0
acid           0
ripe           0
tanni          0
berry          0
citrus         0
chocolate      0
cherry         0
lemon          0
oak            0
white          0
red            0
vanilla        0
herb           0
sweet          0
apple          0
dtype: int64

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.base import BaseEstimator, TransformerMixin

class FrequencyEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns
        self.freq_map = {}
    
    def fit(self, X, y=None):
        for col in self.columns:
            self.freq_map[col] = X[col].value_counts(normalize=True).to_dict()
        return self
    
    def transform(self, X):
        X = X.copy()
        for col in self.columns:
            X[col] = X[col].map(self.freq_map[col])
        return X

preprocessor = ColumnTransformer(
    transformers=[
        ('country', OrdinalEncoder(), ['country']),
        ('province', OneHotEncoder(), ['province']),
        ('region_1', FrequencyEncoder(columns=['region_1']), ['region_1']),
        ('winery', ce.TargetEncoder(), ['winery']),
        ('designation', ce.TargetEncoder(), ['designation']),
        ('price', 'passthrough', ['price']),
        ('points', 'passthrough', ['points']),
        ('rating', 'passthrough', ['rating']),
        ('tropical', 'passthrough', ['tropical']),
        ('fruity', 'passthrough', ['fruity']),
        ('dry', 'passthrough', ['dry']),
        ('acid', 'passthrough', ['acid']),
        ('ripe', 'passthrough', ['ripe']),
        ('tanni', 'passthrough', ['tanni']),
        ('berry', 'passthrough', ['berry']),
        ('citrus', 'passthrough', ['citrus']),
        ('chocolate', 'passthrough', ['chocolate']),
        ('cherry', 'passthrough', ['cherry']),
        ('lemon', 'passthrough', ['lemon']),
        ('oak', 'passthrough', ['oak']),
        ('white', 'passthrough', ['white']),
        ('red', 'passthrough', ['red']),
        ('vanilla', 'passthrough', ['vanilla']),
        ('herb', 'passthrough', ['herb']),
        ('sweet', 'passthrough', ['sweet']),
        ('apple', 'passthrough', ['apple'])
    ]
)

# Define the pipeline
pipe = Pipeline(steps=[('preprocessor', preprocessor)])

# Fit the pipeline on the training data
pipe.fit(X_train, y_train)


joblib.dump(pipe, 'preprocess_pipeline.pkl')

# Transform the test data
X_test= pipe.transform(X_test)

# Transform the train data
X_train= pipe.transform(X_train)

# Transform the val data
X_val= pipe.transform(X_val)



(62153, 26)


In [ ]:
# # Create an instance of the OrdinalEncoder
# le = OrdinalEncoder()

# # Fit the encoder on the 'country' column
# le.fit(X_train['country'])

# # Transform the 'country' column
# X_train['country'] = le.transform(X_train['country'])


In [ ]:
# # Apply one-hot encoding to the 'province' column
# X_train = pd.get_dummies(X_train, columns=['province'])

In [ ]:

# # Create an instance of the TargetEncoder
# encoder = ce.TargetEncoder(cols=['region_1'])

# # Fit the encoder on the training data
# encoder.fit(X_train, y_train)

# # Transform the 'region_1' column in the training and test data
# X_train = encoder.transform(X_train)
# # X_test = encoder.transform(X_test)

In [ ]:
# # Create an instance of the TargetEncoder
# encoder = ce.TargetEncoder(cols=['winery'])

# # Fit the encoder on the training data
# encoder.fit(X_train, y_train)

# # Transform the 'winery' column in the training and test data
# X_train = encoder.transform(X_train)
# # X_test = encoder.transform(X_test)

In [ ]:
# # Create an instance of the TargetEncoder
# encoder = ce.TargetEncoder(cols=['designation'])

# # Fit the encoder on the training data
# encoder.fit(X_train, y_train)

# # Transform the 'winery' column in the training and test data
# X_train = encoder.transform(X_train)
# # X_test = encoder.transform(X_test)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(X_val.shape)

(62153, 52)
(13319, 52)
(13319, 52)


## **Model**

In [ ]:
import re
import optuna
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score



def objective(trial):
    # Suggest values for the hyperparameters
    num_leaves = trial.suggest_int('num_leaves', 2, 4112)
    learning_rate = trial.suggest_float('learning_rate', 1e-8, 1.0)
    n_estimators = trial.suggest_int('n_estimators', 5, 5000)
    min_child_samples = trial.suggest_int('min_child_samples', 3, 1000)
    subsample = trial.suggest_float('subsample', 0.05, 1.0)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.05, 1.0)

    # Create and train the classifier
    clf = lgb.LGBMClassifier(
        num_leaves=num_leaves,
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        min_child_samples=min_child_samples,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        random_state=0
    )
    clf.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=10)

    # Make predictions on the test set
    y_pred = clf.predict(X_val)

    # Compute the accuracy of the classifier
    accuracy = accuracy_score(y_val, y_pred)

     # Report the accuracy to Optuna
    trial.report(accuracy, n_estimators)

    # Handle pruning based on the intermediate value.
    if trial.should_prune():
        raise optuna.TrialPruned()


    return accuracy

In [ ]:
study = optuna.create_study(direction='maximize', pruner=MedianPruner())
study.optimize(objective, n_trials = 20, show_progress_bar=True)
print('Number of finished trials:', len(study.trials))

[I 2023-05-14 05:32:09,596] A new study created in memory with name: no-name-7caed9ed-1012-4366-8cdc-848b4b12810e
/usr/local/lib/python3.10/dist-packages/optuna/progress_bar.py:56: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/20 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	valid_0's multi_logloss: 3.55271
[2]	valid_0's multi_logloss: 4.36369
[3]	valid_0's multi_logloss: 6.64628
[4]	valid_0's multi_logloss: 10.6068
[5]	valid_0's multi_logloss: 25.263
[6]	valid_0's multi_logloss: 31.1234
[7]	valid_0's multi_logloss: 32.8091
[8]	valid_0's multi_logloss: 32.8091
[9]	valid_0's multi_logloss: 32.8091
[10]	valid_0's multi_logloss: 32.8091
[11]	valid_0's multi_logloss: 32.8091
[I 2023-05-14 05:32:15,167] Trial 0 finished with value: 0.35040168180794357 and parameters: {'num_leaves': 3200, 'learning_rate': 0.7152551012708567, 'n_estimators': 3442, 'min_child_samples': 427, 'subsample': 0.6289965507156615, 'colsample_bytree': 0.40872847590602834}. Best is trial 0 with value: 0.35040168180794357.


/usr/local/lib/python3.10/dist-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


...

[1]	valid_0's multi_logloss: 3.22563
[2]	valid_0's multi_logloss: 4.86897
[3]	valid_0's multi_logloss: 7.02297
[4]	valid_0's multi_logloss: 16.5984
[5]	valid_0's multi_logloss: 30.1588
[6]	valid_0's multi_logloss: 33.0243
[7]	valid_0's multi_logloss: 32.988
[8]	valid_0's multi_logloss: 33.1142
[9]	valid_0's multi_logloss: 30.4934
[10]	valid_0's multi_logloss: 30.2275
[11]	valid_0's multi_logloss: 30.356
[I 2023-05-14 05:34:08,148] Trial 9 finished with value: 0.25106990014265335 and parameters: {'num_leaves': 2023, 'learning_rate': 0.7988558172779431, 'n_estimators': 2309, 'min_child_samples': 891, 'subsample': 0.2275632041831247, 'colsample_bytree': 0.07409821630919525}. Best is trial 2 with value: 0.6419400855920114.


/usr/local/lib/python3.10/dist-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	valid_0's multi_logloss: 2.94779
[2]	valid_0's multi_logloss: 2.90946
[3]	valid_0's multi_logloss: 2.86691
[4]	valid_0's multi_logloss: 2.8416
[5]	valid_0's multi_logloss: 2.82254
[6]	valid_0's multi_logloss: 2.79658
[7]	valid_0's multi_logloss: 2.77574
[8]	valid_0's multi_logloss: 2.75728
[9]	valid_0's multi_logloss: 2.72808
[10]	valid_0's multi_logloss: 2.71368
[11]	valid_0's multi_logloss: 2.69595
[12]	valid_0's multi_logloss: 2.67626
[13]	valid_0's multi_logloss: 2.66522
[14]	valid_0's multi_logloss: 2.63862
[15]	valid_0's multi_logloss: 2.61446
[16]	valid_0's multi_logloss: 2.59887
[17]	valid_0's multi_logloss: 2.57931
[18]	valid_0's multi_logloss: 2.5667
[19]	valid_0's multi_logloss: 2.54703
[20]	valid_0's multi_logloss: 2.53722
[21]	valid_0's multi_logloss: 2.52724
[22]	valid_0's multi_logloss: 2.5107
[23]	valid_0's multi_logloss: 2.49853
[24]	valid_0's multi_logloss: 2.4804
[25]	valid_0's multi_logloss: 2.465
[26]	valid_0's multi_logloss: 2.45763
[27]	valid_0's multi_loglos

/usr/local/lib/python3.10/dist-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	valid_0's multi_logloss: 2.98252
[2]	valid_0's multi_logloss: 2.98035
[3]	valid_0's multi_logloss: 2.97357
[4]	valid_0's multi_logloss: 2.96697
[5]	valid_0's multi_logloss: 2.95966
[6]	valid_0's multi_logloss: 2.95432
[7]	valid_0's multi_logloss: 2.94554
[8]	valid_0's multi_logloss: 2.93897
[9]	valid_0's multi_logloss: 2.93242
[10]	valid_0's multi_logloss: 2.92718
[11]	valid_0's multi_logloss: 2.92324
[12]	valid_0's multi_logloss: 2.92124
[13]	valid_0's multi_logloss: 2.91477
[14]	valid_0's multi_logloss: 2.91033
[15]	valid_0's multi_logloss: 2.90671
[16]	valid_0's multi_logloss: 2.90285
[17]	valid_0's multi_logloss: 2.89705
[18]	valid_0's multi_logloss: 2.89256
[19]	valid_0's multi_logloss: 2.88924
[20]	valid_0's multi_logloss: 2.88395
[21]	valid_0's multi_logloss: 2.88196
[22]	valid_0's multi_logloss: 2.87642
[23]	valid_0's multi_logloss: 2.87181
[24]	valid_0's multi_logloss: 2.8676
[25]	valid_0's multi_logloss: 2.86168
[26]	valid_0's multi_logloss: 2.85733
[27]	valid_0's multi_l

In [ ]:
best_params = study.best_params
best_params

In [ ]:
def train_model(X_train, y_train, X_test, y_test, best_params):
    # Create a LightGBM classifier using the best hyperparameters
    clf = lgb.LGBMClassifier(
        num_leaves=best_params['num_leaves'],
        learning_rate=best_params['learning_rate'],
        n_estimators=best_params['n_estimators'],
        min_child_samples=best_params['min_child_samples'],
        subsample=best_params['subsample'],
        colsample_bytree=best_params['colsample_bytree'],
        random_state=0
    )

    # Train the classifier on the training data
    clf.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=10)

    # Make predictions on the test data
    y_pred = clf.predict(X_test)

    # Compute the accuracy of the classifier
    accuracy = accuracy_score(y_test, y_pred)

    return clf, accuracy

# Train the final model using the best hyperparameters
clf, accuracy = train_model(X_train, y_train, X_test, y_test, best_params)

# Print the accuracy of the final model
print(f'Accuracy: {accuracy:.2f}')

In [ ]:
import joblib

clf.fit(X_val, y_val)
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

# Save the model using joblib
joblib.dump(clf, 'lgbm_model.joblib')

# Save the model using the built-in save_model method
clf.booster_.save_model('lgbm_model.txt')